In [355]:
import nltk
from nltk.corpus import wordnet

import importlib
import pandas as pd
import re

import utils
importlib.reload(utils)

import utils

[nltk_data] Downloading package punkt to C:\Users\Lenovo Thinkpad
[nltk_data]     T430\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Lenovo Thinkpad
[nltk_data]     T430\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Lenovo Thinkpad
[nltk_data]     T430\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Lenovo Thinkpad
[nltk_data]     T430\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Lenovo Thinkpad
[nltk_data]     T430\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##### Load model

In [356]:
model, tokenizer = utils.load_model(debug=True)

loading configuration file config.json from cache at C:\Users\Lenovo Thinkpad T430\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6\snapshots\a4f8f3ea906ed274767e9906dbaede7531d660ff\config.json
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_

##### Preprocess text function

##### Summarize text function

##### Function to summarize in chunks

##### Function to detect valid summary part

In [357]:
utils.is_in_enchant_dict("antipsychotic")

True

In [358]:
# List of valid English words for reference
valid_words = set(nltk.corpus.words.words())

# List of valid English words for reference
valid_words = set(nltk.corpus.words.words())

domain_specific_words = ['lamotrigine', 'antidepressant', 'placebo', 'monotherapy']
valid_words.update(domain_specific_words)

print(list(valid_words)[0])

archswindler


###### Check NLTK pos tags

In [359]:
nltk.pos_tag(['lamotrigine', 'evaluated'])

[('lamotrigine', 'NN'), ('evaluated', 'VBD')]

##### Expand contractions

In [360]:
# check expland contractions
check_text = "also known is its ability to improve social and occupational functioning of patients who are not taking it if they don't feel well "

utils.expand_contractions(check_text)

'also known is its ability to improve social and occupational functioning of patients who are not taking it if they do not feel well '

##### Clean summary function

In [361]:
check_sentence = "lamotrigine was evaluated for its antidepressant efficacy and safety"
print(f"\n----- {check_sentence}")

tokens = nltk.word_tokenize(check_sentence)
print(f"\n----- {tokens}")

pos_tags = nltk.pos_tag(tokens)
print(f"\n----- {pos_tags}")

cleaned_tokens = []
for token, pos_tag in pos_tags:
    # Check if the lemmatized form of the token is valid
    if utils.is_valid_word(valid_words, token, pos_tag):
        cleaned_tokens.append(token)
    else:
        # Once gibberish or non-valid words appear, stop processing
        break
print(f"\n Cleaned tokens: {cleaned_tokens}")



----- lamotrigine was evaluated for its antidepressant efficacy and safety

----- ['lamotrigine', 'was', 'evaluated', 'for', 'its', 'antidepressant', 'efficacy', 'and', 'safety']

----- [('lamotrigine', 'NN'), ('was', 'VBD'), ('evaluated', 'VBN'), ('for', 'IN'), ('its', 'PRP$'), ('antidepressant', 'JJ'), ('efficacy', 'NN'), ('and', 'CC'), ('safety', 'NN')]

 Cleaned tokens: ['lamotrigine', 'was', 'evaluated', 'for', 'its', 'antidepressant', 'efficacy', 'and', 'safety']


##### Detect valid words model

In [362]:
valid_words_df = pd.read_csv("input_files/valid_words_labelled.csv")

# create features
valid_words_df['contains_number'] = valid_words_df['word'].str.contains(r'\d').astype(int)
valid_words_df['num_dashes'] = valid_words_df['word'].str.count('-')
valid_words_df['num_vowels'] = valid_words_df['word'].str.count(r'[aeiouAEIOU]')
valid_words_df['contains_non_alphanum'] = valid_words_df['word'].str.contains(r'[^a-zA-Z0-9-]').astype(int)
valid_words_df['contains_ine'] = valid_words_df['word'].str.contains(r'ine', case=False).astype(int)

pos_tags = valid_words_df['word'].apply(lambda x: nltk.pos_tag([x])[0])
valid_words_df['lemma'] = pos_tags.apply(lambda x: utils.lemmatize_token(x[0], x[1]))
valid_words_df['is_real_word_enchant'] = valid_words_df['lemma'].apply(utils.is_in_enchant_dict).astype(int)

valid_words_df['num_characters'] = valid_words_df['word'].apply(len)
# Create the 'starts_with_letter' feature
valid_words_df['starts_with_letter'] = valid_words_df['word'].apply(lambda x: int(bool(re.match(r'^[A-Za-z]', x))))
valid_words_df['ends_with_letter'] = valid_words_df['word'].apply(lambda x: int(bool(re.search(r'[A-Za-z]$', x))))

# List of medical terms to check
medical_terms = ['-mg', 'dsm', 'dsm-', 'dsm-5', 'dsm-iii', 'cochrane', 'pdd', 'al.', 'prisma', 'asberg', 'cgi', 'qigong', 'subscale', 'waitlist', 'anxiolytic']  # Extend as needed

# Function to check if a string contains any of the medical terms
def contains_medical_term(word):
    return any(term in word for term in medical_terms)

valid_words_df['contains_medical_term'] = valid_words_df['word'].apply(contains_medical_term).astype(int)

print(f"\nLen dataset for model: {len(valid_words_df.index)}")
print(f"\n{valid_words_df.columns}")
print(f"\n----- {pos_tags[:15]}")
print("\n")
print( valid_words_df.head(20) )


Len dataset for model: 177

Index(['word', 'valid_label', 'contains_number', 'num_dashes', 'num_vowels',
       'contains_non_alphanum', 'contains_ine', 'lemma',
       'is_real_word_enchant', 'num_characters', 'starts_with_letter',
       'ends_with_letter', 'contains_medical_term'],
      dtype='object')

----- 0           ((versiune, NN)
1           (2001–2005, CD)
2                 (25-, JJ)
3           (31absence, CD)
4               (50-mg, JJ)
5                 (50s, CD)
6                (5th, NNS)
7                   (â, NN)
8           (adjuctive, JJ)
9                 (al., NN)
10                (am:, NN)
11           (american, JJ)
12              (andor, NN)
13         (anecyolypi, NN)
14    (antipsychotics, NNS)
Name: word, dtype: object


               word  valid_label  contains_number  num_dashes  num_vowels  \
0         (versiune            0                0           0           4   
1         2001–2005            0                1           0           0   
2    

##### Valid word classifier

In [363]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

In [364]:
# Separate features (X) and target (y)
X = valid_words_df.drop(columns=['valid_label', 'word', 'lemma'])
y = valid_words_df['valid_label']

print("\n-----")
print(X.columns)
print(len(X.index))
print(len(y))

# Identify binary and non-binary features
binary_features = ['contains_number', 'contains_non_alphanum', 'contains_ine', 'ends_with_letter', 'starts_with_letter', 'is_real_word_enchant', 'contains_medical_term']
continuous_features = ['num_vowels', 'num_characters', 'num_dashes']
print(f"\nNum features: binary {len(binary_features)} + numeric {len(continuous_features)}")


-----
Index(['contains_number', 'num_dashes', 'num_vowels', 'contains_non_alphanum',
       'contains_ine', 'is_real_word_enchant', 'num_characters',
       'starts_with_letter', 'ends_with_letter', 'contains_medical_term'],
      dtype='object')
177
177

Num features: binary 7 + numeric 3


In [365]:
# Preprocessing: Standardize continuous features
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), continuous_features),
        ('passthrough', 'passthrough', binary_features)  # Leave binary features as-is
    ]
)

# Create a pipeline with preprocessing and logistic regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, C=1.0))  # Adjust C for regularization strength
])

# Perform cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=10, scoring='accuracy')

print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Accuracy Scores: [1.         0.94444444 0.88888889 0.94444444 0.77777778 0.94444444
 1.         0.88235294 1.         1.        ]
Mean Accuracy: 0.9382352941176471


In [366]:
# Perform cross-validation and get predictions
y_pred = cross_val_predict(pipeline, X, y, cv=10)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Identify false negatives (FN) and false positives (FP)
FN_indices = (y == 1) & (y_pred == 0)  # True label is 1, predicted as 0
FP_indices = (y == 0) & (y_pred == 1)  # True label is 0, predicted as 1

# Extract examples of FN and FP from the original dataframe
FN_examples = valid_words_df[FN_indices]
FP_examples = valid_words_df[FP_indices]

# Print false negative examples
print("\nFalse Negatives (FN):")
print(FN_examples[['word', 'valid_label']])  # Adjust columns as needed

# Print false positive examples
print("\nFalse Positives (FP):")
print(FP_examples[['word', 'valid_label']])  # Adjust columns as needed

Confusion Matrix:
 [[81  7]
 [ 4 85]]

False Negatives (FN):
          word  valid_label
51       dsm-5            1
52     dsm-iii            1
81  interrater            1
98          mg            1

False Positives (FP):
             word  valid_label
13     anecyolypi            0
70             gi            0
72        goodwin            0
79             ii            0
82   inventory-ii            0
121          ovid            0
149         rocha            0


###### Run model on whole dataset

In [367]:
# Fit the pipeline on the entire dataset
pipeline.fit(X, y)

# Predict the labels for the entire dataset
y_pred = pipeline.predict(X)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y, y_pred)
print("\nComplete Dataset Confusion Matrix:\n", conf_matrix)

# Identify false negatives (FN) and false positives (FP)
FN_indices = (y == 1) & (y_pred == 0)  # True label is 1, predicted as 0
FP_indices = (y == 0) & (y_pred == 1)  # True label is 0, predicted as 1

# Extract examples of FN and FP from the original dataframe
FN_examples = valid_words_df[FN_indices]
FP_examples = valid_words_df[FP_indices]

# Print false negative examples
print("\nFalse Negatives (FN):")
print(FN_examples[['word', 'valid_label']])  # Adjust columns as needed

# Print false positive examples
print("\nFalse Positives (FP):")
print(FP_examples[['word', 'valid_label']])  # Adjust columns as needed


Complete Dataset Confusion Matrix:
 [[80  8]
 [ 2 87]]

False Negatives (FN):
       word  valid_label
51    dsm-5            1
52  dsm-iii            1

False Positives (FP):
             word  valid_label
13     anecyolypi            0
70             gi            0
72        goodwin            0
79             ii            0
82   inventory-ii            0
121          ovid            0
149         rocha            0
157   sportdiscus            0


##### SVM

In [368]:
# Create pipeline with SVM (RBF kernel) using the existing preprocessor
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Reuse existing preprocessor
    ('classifier', SVC(kernel='rbf', C=1.0, gamma='scale', probability=True))
])

# Perform cross-validation (accuracy)
svm_cv_scores = cross_val_score(svm_pipeline, X, y, cv=10, scoring='accuracy')
print("SVM Cross-Validation Accuracy Scores:", svm_cv_scores)
print("SVM Mean Accuracy:", svm_cv_scores.mean())

SVM Cross-Validation Accuracy Scores: [0.77777778 0.88888889 0.88888889 0.94444444 0.83333333 1.
 1.         0.76470588 0.94117647 1.        ]
SVM Mean Accuracy: 0.903921568627451


In [369]:
# Perform cross-validation predictions
y_pred_svm = cross_val_predict(svm_pipeline, X, y, cv=10)

# Evaluate performance using confusion matrix and classification report
svm_conf_matrix = confusion_matrix(y, y_pred_svm)
print("\nSVM Confusion Matrix:\n", svm_conf_matrix)

svm_class_report = classification_report(y, y_pred_svm, target_names=['Class 0', 'Class 1'])
print("\nSVM Classification Report:\n", svm_class_report)

# Identify false negatives (FN) and false positives (FP)
FN_indices = (y == 1) & (y_pred == 0)  # True label is 1, predicted as 0
FP_indices = (y == 0) & (y_pred == 1)  # True label is 0, predicted as 1

# Extract examples of FN and FP from the original dataframe
FN_examples = valid_words_df[FN_indices]
FP_examples = valid_words_df[FP_indices]

# Print false negative examples
print("\nFalse Negatives (FN):")
print(FN_examples[['word', 'valid_label']])  # Adjust columns as needed

# Print false positive examples
print("\nFalse Positives (FP):")
print(FP_examples[['word', 'valid_label']])  # Adjust columns as needed


SVM Confusion Matrix:
 [[79  9]
 [ 8 81]]

SVM Classification Report:
               precision    recall  f1-score   support

     Class 0       0.91      0.90      0.90        88
     Class 1       0.90      0.91      0.91        89

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.90      0.90      0.90       177


False Negatives (FN):
       word  valid_label
51    dsm-5            1
52  dsm-iii            1

False Positives (FP):
             word  valid_label
13     anecyolypi            0
70             gi            0
72        goodwin            0
79             ii            0
82   inventory-ii            0
121          ovid            0
149         rocha            0
157   sportdiscus            0


##### Function to create features for token

##### Classifier model integration

##### Clean summary function

##### Functions to import text

In [370]:
import os

# Define the path to the folder containing the PDFs
pdf_folder = 'pdf_files'

# List all files in the pdf_folder and filter to include only PDFs
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
print(pdf_files)

['NPR2-42-120.pdf']


In [371]:
prompts = [
    "Effects of lamotrigine on unipolar depression.",
    "Impact of lamotrigine on unipolar depression.",
    "Key findings related to lamotrigine in treating unipolar depression.",
    "Outcomes and statistics related to lamotrigine and unipolar depression."
]

In [372]:
# Initialize a list to store the summaries
summaries = []

In [373]:
pdf_folder = "pdf_files"  # Replace with your actual folder name

# Select the first PDF file and process it
ind = 0
curr_file_path = os.path.join(pdf_folder, pdf_files[ind])  # First file
print(f"Current file: {curr_file_path}")

# Open the PDF and extract text page-by-page
with open(curr_file_path, 'rb') as file:
    pages_text = utils.read_pdf_by_page(file)  # Get text for each page separately

# Save each page's text to separate files (optional)
for i, page_text in enumerate(pages_text):
    with open(f"page_{i + 1}_text.txt", "w", encoding="utf-8") as text_file:
        text_file.write(page_text)

# print('\n', curr_file_text)
# print('\n', extracted_text)

Current file: pdf_files\NPR2-42-120.pdf
Extracted text for Page 1:
 Received: 17 April 2021 | Revised: 12 November 2021 | Accepted: 15 December 2021DOI: 10.1002/npr2.12228CASE REPORTThe effectiveness of lamotrigine for persistent depressivedisorder: A case reportYusuke Matsuzaka1,2 | Kayoko Urashima1,2 | Shintaro Sakai1 | Yoshiro Morimoto1,2 |Shinji Kanegae1 | Hirohisa Kinoshita1,3 | Akira Imamura1,4 | Hiroki Ozawa1,21Department of Neuropsychiatry,Nagasaki University Hospital, Nagasaki, AbstractJapan Aim: Persistent depressive disorder (PDD) was first introduced in the Diagnostic and2Department of Neuropsychiatry, Statistical Manual of Mental Disorders 5th edition (DSM-5), which encompasses nu-Nagasaki University Graduate School ofBiomedical Sciences, Nagasaki, Japan merous different conditions, including dysthymia, recurrent major depressive disor-3Health Center, Nagasaki University, der, double depression, and chronic major depression. SSRIs are the first-line drugs forNagasaki, Japa

#### Load txt files - processing start

In [374]:
txt_folder = 'txt_files'

txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]
print(txt_files)

['Adel_Gabriel_Lamotrigine_2006.txt', 'exercise_depression_2024_au.txt', 'matsuzaka_lamotrigine_2021.txt']


##### Choose text file

In [375]:
ind = 1
curr_file = os.path.join(txt_folder, txt_files[ind])
print(f"\nCurrent file: {curr_file} \n")
with open(curr_file, 'r', encoding='utf-8') as file:
    # file_content = read_txt(file)
    file_content = file.read()

print(file_content)


Current file: txt_files\exercise_depression_2024_au.txt 

Introduction
Major depressive disorder is a leading cause of
disability worldwide and has been found to lower life
satisfaction more than debt, divorce, and diabetes and to exacerbate comorbidities, including heart
disease, anxiety, and cancer. Although people with
major depressive disorder often respond well to drug
treatments and psychotherapy, many are resistant to
treatment. In addition, access to treatment for many
people with depression is limited, with only 51%
treatment coverage for high income countries and 20%
for low and lower-middle income countries. More
evidence based treatments are therefore needed.
Exercise may be an effective complement or
alternative to drugs and psychotherapy. In addition
to mental health benefits, exercise also improves a
range of physical and cognitive outcomes. Clinical
practice guidelines in the US, UK, and Australia
recommend physical activity as part of treatment for
depression. But the

In [376]:
# Apply to your file content
num_tokens = utils.count_tokens(file_content, tokenizer)

print(f"The number of tokens in the text: {num_tokens}")

Token indices sequence length is longer than the specified maximum sequence length for this model (3286 > 1024). Running this sequence through the model will result in indexing errors


The number of tokens in the text: 3286


##### Test specific chunks

In [377]:
chunks = utils.split_text_into_chunks(file_content, tokenizer)
print(f"\n Num chunks: {len(chunks)}")
chunk_ind = 0
print("\n----------")
# print(chunks[chunk_ind])

print("\n----------")
chunk_processed = utils.preprocess_text(chunks[chunk_ind])
print( chunk_processed )


 Num chunks: 4

----------

----------
Introduction Major depressive disorder is a leading cause of disability worldwide and has been found to lower life satisfaction more than debt, divorce, and diabetes and to exacerbate comorbidities, including heart disease, anxiety, and cancer. Although people with major depressive disorder often respond well to drug treatments and psychotherapy, many are resistant to treatment. In addition, access to treatment for many people with depression is limited, with only 51 treatment coverage for high income countries and 20 for low and lower-middle income countries. More evidence based treatments are therefore needed. Exercise may be an effective complement or alternative to drugs and psychotherapy. In addition to mental health benefits, exercise also improves a range of physical and cognitive outcomes. Clinical practice guidelines in the US, UK, and Australia recommend physical activity as part of treatment for depression. But these guidelines do not 

In [378]:
utils.get_non_identified_words(valid_words, file_content)

['hypothesise',
 'findingsfor',
 'worldwide',
 'programmes',
 'ssris',
 'zealand',
 'favourable',
 'ovid',
 'prisma',
 'personalising',
 'controlled',
 'baseline',
 'metaanalyses-network',
 'qigong',
 'nuanced',
 'meta-analysesthat',
 'online',
 'behavioural',
 'psycinfo',
 'practised',
 'database',
 'eg',
 'others',
 'κ71',
 'sportdiscus',
 'κ15',
 'httpsosf.ionzw6u',
 'metaanalysis',
 'et',
 'inventory-ii',
 'heissel',
 'british',
 'prisma-nma',
 'practise',
 'existing',
 'medline',
 'comorbidities',
 'assessmentsfor',
 'uk',
 'australian',
 'keywords',
 'cochrane',
 'american',
 'postpartum',
 'embase',
 'reuptake',
 'randomised',
 'australia',
 's2',
 's3',
 'neurobiological',
 's0',
 'predefined',
 'waitlist',
 'serotonin']

###### Summarize text check

In [ ]:
summarized_text = utils.summarize_text(chunk_processed, tokenizer, 200, 700, prompts, print_num_tokens=True)
print(summarized_text)

NameError: name 'tokenizer' is not defined

In [122]:
clean_summary(summarized_text)

'a total of 218 studies described in 246 reports were included , totalling 495 arms and 14170 participants . the most effective exercise modalities were walking or jogging , yoga , strength training , and dancing . to be eligible for inclusion , studies had to be.'

In [87]:
summarize_long_text(chunks[chunk_ind], 200, 700, prompts)

'a total of 218 studies described in 246 reports were included , totalling 495 arms and 14170 participants . the most effective exercise modalities were walking or jogging , yoga , strength training , and dancing . to be eligible for inclusion , studies had to be.'

##### Summarize whole article

In [119]:
final_summary = summarize_long_text(file_content, 200, 700, prompts)

base_filename = re.sub(r".*/(.*)\.txt$", r"\1", curr_file)

final_text = f"{base_filename}\n\n{final_summary}"
print(final_text)

output_filename = f"{base_filename}_summary.txt"

# Write the final text to a text file
with open(output_filename, "w") as text_file:
    text_file.write(final_text)

print(f"Summary saved to {output_filename}")

exercise_depression_2024_au

major depressive disorder is a leading cause of disability worldwide . access to treatment for many people with depression is limited . exercise may be an effective complement or alternative to drugs and psychotherapy . findings are presented according to the Preferred Reporting Items for Systematic Reviews. a total of 218 studies described in 246 reports were included , totalling 495 arms and 14170 participants . the most effective exercise modalities were walking or jogging , yoga , strength training , and dancing . to be eligible for inclusion , studies had to be. our review did not uncover clear causal mechanisms , but the trends in the data are useful for generating hypotheses . we. the world health organization recommends physical activity for everyone . many patients may have physical , psychological , or social barriers to participation . effective exercise modalities could be considered as core treatments for depression . but not all patients can a